In [1]:
import configparser
from fastai.vision import *

In [2]:
config = configparser.ConfigParser()
config.read('config/conf.ini')

['config/conf.ini']

In [3]:
model = load_learner(config['CONF']['model_folder'], config['CONF']['model_name'])

In [4]:
# TODO properties existance, validation..
input_folder = config['CONF']['input_folder']
output_folder = config['CONF']['output_folder']

for i in os.listdir(input_folder):
    if i.endswith(".jpg"): # TODO png?
        img = open_image(os.path.join(input_folder, i))
        img_title, img_extension = os.path.splitext(i)
        pred_class, pred_idx, outputs = model.predict(img)
        prediction = (str(pred_class), round(max(outputs.numpy()) * 100))
        
        # TODO according to pred_class do rotation (90,180,270)
        
        path_to_class = output_folder + "/" + str(pred_class)
        # TODO find all categories from model prehand..
        if not os.path.exists(path_to_class):
            os.makedirs(path_to_class)
        
        new_img_title = img_title + "_" + prediction[0] + "_" + str(prediction[1]) + img_extension
        new_img_path = os.path.join(path_to_class, new_img_title)
        img.save(new_img_path)